# Image Classification using Support Vector Machine (SVM) 

## Description
This code performs image classification using Support Vector Machine (SVM) with HOG features.
It preprocesses cat and dog images, extracts HOG features, trains the SVM model, and evaluates classification performance.
The code saves the trained model and provides metrics like precision, recall, accuracy, and a classification report.

In [1]:
from pre_process import PreProcess # Import your custom PreProcess class
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import io, transform
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import joblib

In [2]:
# Initialize the object for custom class
process = PreProcess()

### Get the Labels of our Cat and Dog Images Data

In [3]:
cat_path = "D:\\New folder\\Data\\train\\cats"  # Specify the path to cat images
dog_path = "D:\\New folder\\Data\\train\\dogs"  # Specify the path to dog images
cat_df, dog_df = process.file_labelling(cat_path, dog_path)  # Label the image files as cat and dog

In [4]:
cat_df.head() # Display the first few rows of the cat data

,file,target
0,D:\New folder\Data\train\cats\cat.0.jpg,0
1,D:\New folder\Data\train\cats\cat.1.jpg,0
2,D:\New folder\Data\train\cats\cat.100.jpg,0
3,D:\New folder\Data\train\cats\cat.1000.jpg,0
4,D:\New folder\Data\train\cats\cat.10002.jpg,0


In [5]:
dog_df.head()  # Display the first few rows of the dog data

,file,target
0,D:\New folder\Data\train\dogs\dog.0.jpg,1
1,D:\New folder\Data\train\dogs\dog.1.jpg,1
2,D:\New folder\Data\train\dogs\dog.10.jpg,1
3,D:\New folder\Data\train\dogs\dog.1000.jpg,1
4,D:\New folder\Data\train\dogs\dog.10000.jpg,1


### Initialize empty lists and dataframes to store HOG features

In [6]:
ls_hog = []
count = 0
df_cat = pd.DataFrame()

# Extract HOG features for a subset of cat images (the first 5000)
total_iterations = 0

for count, image_path in enumerate(cat_df['file'].loc[:5000]):
    extracted_feature, img_path = process.extract_hog_feature(image_path)  # Extract HOG features for an image
    ls_hog.append([extracted_feature, img_path])  # Store HOG feature and image path

total_iterations = count + 1  # Calculate the total count of iterations

print("Total iterations:", total_iterations)

Total iterations: 5001


In [7]:
# Concatenate the HOG features to create a dataframe
for i in range(len(ls_hog)):
    df_cat = pd.concat([df_cat, pd.Series(ls_hog[i][0])], axis=1)

In [8]:
# Doing tranpose to convert the row into column and column into row
df_upd_cat = df_cat.transpose()

In [9]:
# Now the index will start from 0 to n
df_upd_cat = df_upd_cat.reset_index()

In [10]:
# Display the first few rows of the transposed cat HOG features dataframe
df_upd_cat.head()

,index,0,1,2,3,4,5,6,7,8,...,8090,8091,8092,8093,8094,8095,8096,8097,8098,8099
0,0,0.347284,0.000000,0.000000,0.000000,0.000833,0.024801,0.098172,0.096082,0.347284,...,0.053916,0.033221,0.025633,0.092108,0.137179,0.274561,0.040702,0.000000,0.000000,0.013685
1,0,0.254146,0.059052,0.032694,0.056351,0.070183,0.041041,0.206986,0.176438,0.254146,...,0.113792,0.245764,0.136152,0.237635,0.153505,0.199295,0.159452,0.107791,0.030850,0.102147
2,0,0.096934,0.083190,0.024875,0.114956,0.128810,0.081335,0.053667,0.186580,0.094006,...,0.256743,0.162735,0.011663,0.000000,0.000000,0.011753,0.004072,0.001272,0.040142,0.429850
3,0,0.102091,0.083605,0.124746,0.173507,0.255841,0.219440,0.132731,0.091359,0.000000,...,0.077533,0.232927,0.232927,0.165472,0.022227,0.232927,0.196072,0.186619,0.041276,0.071166
4,0,0.151428,0.000000,0.041891,0.128357,0.302971,0.302971,0.302971,0.083599,0.028436,...,0.041595,0.257501,0.257501,0.132011,0.094984,0.180869,0.013782,0.001558,0.116869,0.180745


### Extract HOG features for a subset of dog images (the first 5000)

In [12]:
ls_hog_dog = []  # Initialize the ls_hog_dog list

total_iterations = 0

for count, image_path in enumerate(dog_df['file'].loc[:5000]):
    extracted_feature_dog, image_path = process.extract_hog_feature(image_path)  # Extract HOG features for a dog image
    ls_hog_dog.append([extracted_feature_dog, image_path])  # Store HOG feature and image path

total_iterations = count + 1  # Calculate the total count of iterations

print("Total iterations:", total_iterations)

Total iterations: 5001


In [14]:
df_dog = pd.DataFrame()
# Concatenate the HOG features to create a dataframe
for i in range (len(ls_hog_dog)):
    df_dog = pd.concat([df_dog, pd.Series(ls_hog_dog[i][0])],axis=1)

In [15]:
# Doing tranpose to convert the row into column and column into row
df_upd_dog = df_dog.transpose()

In [16]:
# Reset the index of the dataframe and Now the index will start from 0 to n 
df_upd_dog = df_upd_dog.reset_index()

In [17]:
# Drop the 'index' column from both dataframes
df_upd_cat.drop('index', axis=1, inplace=True)
df_upd_dog.drop('index', axis=1, inplace=True)

In [18]:
# Add a 'target' column to indicate cat (0) or dog (1)
df_upd_cat['target'] = 0
df_upd_dog['target'] = 1

### Concatenate cat and dog dataframes to create the final dataset

In [19]:
final_data = pd.concat([df_upd_cat, df_upd_dog], axis=0, ignore_index=True)

In [20]:
final_data.shape  # Display the shape of the final dataset

(10002, 8101)

In [21]:
# Saving the final_data in csv format
final_data.to_csv('output.csv', index=False)

### Split the dataset into training and testing sets

In [22]:
X_train, X_test, y_train, y_test = process.split_dataset(final_data)

In [23]:
# Using MinMaxScaler 
scaler = MinMaxScaler()

In [24]:
# Apply MinMaxScalr on train and test data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Initialize a Support Vector Machine (SVM) classifier

In [25]:
best_svm = SVC()
best_svm.fit(X_train, y_train)

SVC()

### Save the trained SVM model to a file

In [26]:
model_filename = 'best_svm_model.joblib'
joblib.dump(best_svm, model_filename)

['best_svm_model.joblib']

In [27]:
# Predict the labels for the test data
y_pred = best_svm.predict(X_test)

# Visualize the Model Performance

In [28]:
# Calculate precision
precision = precision_score(y_test, y_pred)
print("Precision:", precision)

Precision: 0.7431906614785992


In [29]:
# Calculate recall
recall = recall_score(y_test, y_pred)
print("Recall:", recall)

Recall: 0.7453658536585366


In [30]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7376311844077961


In [31]:
# Calculate F1-score
f1 = f1_score(y_test, y_pred)
print("F1-score:", f1)


F1-score: 0.7442766682903069


In [32]:
# Calculate confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[712 264]
 [261 764]]


In [33]:
# Generate a classification report and print it
print("Classification Report:")
print(classification_report(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.73      0.73       976
           1       0.74      0.75      0.74      1025

    accuracy                           0.74      2001
   macro avg       0.74      0.74      0.74      2001
weighted avg       0.74      0.74      0.74      2001



# Thankyou